In [9]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.models as models
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Dataset
from PIL import Image
import os
import pandas as pd


In [ ]:
import kagglehub
# Download latest version
path = kagglehub.dataset_download("xhlulu/vinbigdata-chest-xray-png-512px-original-ratio")
#path = kagglehub.dataset_download("xhlulu/vinbigdata-chest-xray-resized-png-256x256")

print("Path to dataset files:", path)

Resuming download from 185597952 bytes (1639501952 bytes left)...
Resuming download from https://www.kaggle.com/api/v1/datasets/download/xhlulu/vinbigdata-chest-xray-png-512px-original-ratio?dataset_version_number=1 (185597952/1825099904) bytes left.


100%|██████████| 1.70G/1.70G [01:47<00:00, 15.3MB/s]

Extracting files...


Path to dataset files: C:\Users\jsayed\.cache\kagglehub\datasets\xhlulu\vinbigdata-chest-xray-png-512px-original-ratio\versions\1
len of path: 106


In [11]:
import torch
print(torch.__version__)
print(torch.version.cuda)
print(torch.backends.cudnn.version())

print("CUDA Available:", torch.cuda.is_available())
print("Device Count:", torch.cuda.device_count())
print("GPU Name:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "No GPU detected")
print("Current Device:", torch.cuda.current_device() if torch.cuda.is_available() else "No GPU detected")
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")
print()
print()
for i in range(torch.cuda.device_count()):
    print(f"GPU {i}: {torch.cuda.get_device_name(i)}")


2.6.0+cpu
None
None
CUDA Available: False
Device Count: 0
GPU Name: No GPU detected
Current Device: No GPU detected
Using device: cpu




In [2]:
# Load pre-trained ResNet models
resnet18 = models.resnet18(pretrained=True)
resnet34 = models.resnet34(pretrained=True)
resnet50 = models.resnet50(pretrained=True)

# Remove classification head (fully connected layer)
for model in [resnet18, resnet34, resnet50]:
    model.fc = nn.Identity()  # Output raw features instead of class scores
    model.eval()  # Set to evaluation mode

# Move models to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
resnet18, resnet34, resnet50 = resnet18.to(device), resnet34.to(device), resnet50.to(device)


C:\Users\jsayed\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\jsayed\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to C:\Users\jsayed/.cache\torch\hub\checkpoints\resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:0

cpu


In [16]:
def extract_features(models, image):
    with torch.no_grad():  # Disable gradients for efficiency
        features = [model(image).flatten(start_dim=1) for model in models]  # Extract feature vectors
        return torch.cat(features, dim=1)  # Concatenate all feature vectors

meta_learner = nn.Linear(512 + 512 + 2048, 2).to(device)  # 2 output classes (e.g., normal vs disease)


See X num of files from train set for labelling purpose and send labels to new csv

In [ ]:
# PARAMETERS
X = 100  # Number of files to extract
train_folder = r"C:\Users\jsayed\Downloads\DHBW\lung-disease-detection\dataset\train"  # Path to your train dataset folder
csv_file = r"C:\Users\jsayed\Downloads\DHBW\lung-disease-detection\dataset\train.csv"  # Path to your full train.csv file
output_csv = rf"C:\Users\jsayed\Downloads\DHBW\lung-disease-detection\dataset\filtered_train_{X}.csv"  # Output CSV file

# STEP 1: Get the first X UNIQUE filenames from the dataset folder
all_files = sorted(os.listdir(train_folder))  # Sort for consistency

# Extract unique filenames (remove extensions like .dicom)
unique_files = set()
selected_files = []

for file in all_files:
    file_base = os.path.splitext(file)[0]  # Remove extension
    if file_base not in unique_files:  # Ensure uniqueness
        unique_files.add(file_base)
        selected_files.append(file_base)
    if len(selected_files) == X:  # Stop when we reach X unique files
        break

# STEP 2: Load the train.csv file
df = pd.read_csv(csv_file)

# STEP 3: Filter the CSV file to only include selected filenames
filtered_df = df[df['image_id'].isin(selected_files)]  # Assuming 'filename' is the column name

# STEP 4: Save the filtered CSV
filtered_df.to_csv(output_csv, index=False)

print(f"Filtered dataset with {len(filtered_df)} entries saved to {output_csv}")


Filtered dataset with 4424 entries saved to C:\Users\jsayed\Downloads\DHBW\lung-disease-detection\dataset\filtered_train_1000.csv


In [17]:
class LungXrayDataset(Dataset):
    def __init__(self, csv_file, img_dir, transform=None):
        """
        Args:
            csv_file (str): Path to the CSV file with image filenames & labels.
            img_dir (str): Path to the folder containing images.
            transform (callable, optional): Optional transform to be applied.
        """
        self.data = pd.read_csv(csv_file)
        self.img_dir = img_dir
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        # Get filename and label
        img_name = os.path.join(self.img_dir, self.data.iloc[idx, 0])  # Column 0 = filename
        label = int(self.data.iloc[idx, 1])  # Column 1 = label

        # Open image
        image = Image.open(img_name).convert("RGB")  # Ensure it's in RGB format

        # Apply transformations
        if self.transform:
            image = self.transform(image)

        return image, label  # Return the image tensor and label


In [20]:
# Define Image Transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize images to 224x224
    transforms.ToTensor(),  # Convert to PyTorch tensor
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])  # Normalize to [-1,1] range
])


# Paths
csv_path = csv_file  # CSV file with labels
img_folder = train_folder  # Folder where images are stored

# Create dataset
dataset = LungXrayDataset(csv_path, img_folder, transform=transform)

# Create DataLoader for batching
dataloader = DataLoader(dataset, batch_size=32, shuffle=True, num_workers=4)

# Check dataset size
print(f"Dataset contains {len(dataset)} images")



Dataset contains 67914 images


In [ ]:
# Get one batch of images & labels
images, labels = next(iter(dataloader))

# Check tensor shapes
print(f"Image batch shape: {images.shape}")  # Should be (batch_size, 3, 224, 224)
print(f"Labels batch shape: {labels.shape}")  # Should be (batch_size,)


In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(meta_learner.parameters(), lr=0.001)

# Training Loop
for epoch in range(5):  # Train for 5 epochs
    for images, labels in dataloader:
        images, labels = images.to(device), labels.to(device)

        # Step 1: Extract features from ResNet models
        features = extract_features([resnet18, resnet34, resnet50], images)

        # Step 2: Pass features through the meta-learner
        outputs = meta_learner(features)
        loss = criterion(outputs, labels)

        # Step 3: Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print(f"Epoch [{epoch+1}/5], Loss: {loss.item():.4f}")


In [ ]:
def predict(image):
    image = transform(image).unsqueeze(0).to(device)  # Prepare image
    features = extract_features([resnet18, resnet34, resnet50], image)  # Get features
    output = meta_learner(features)  # Predict
    return torch.argmax(output, dim=1).item()  # Return class label
